In [1]:
""" Load some libs """
""" python 2 lib using networkx """
import matplotlib.pyplot as plt
import networkx as nx
import random
import math
import pandas as pd
import statsmodels.api as sm
import glob
import os
import numpy as np
from PIL import Image
from helpers import *
import pickle
import time
#random.seed(100)
#tic = time.time()

/home/lucaskruitwagen/miniconda3/envs/py2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
""" Let's see what WEPPS we have """
folders = sorted(glob.glob('WEPP_FILES/*'))
print folders


['WEPP_FILES/2004-Q4-DEC', 'WEPP_FILES/2007-Q4-DEC', 'WEPP_FILES/2008-Q4-DEC', 'WEPP_FILES/2009-Q4-DEC', 'WEPP_FILES/2010-Q4-DEC', 'WEPP_FILES/2011-Q3-SEP', 'WEPP_FILES/2012-Q4-DEC', 'WEPP_FILES/2013-Q4-DEC', 'WEPP_FILES/2014-q4-dec', 'WEPP_FILES/2015-Q1-MAR', 'WEPP_FILES/2015-Q2-JUN', 'WEPP_FILES/2015-Q4-DEC', 'WEPP_FILES/2015-q3-sep', 'WEPP_FILES/2016-Q1-APR', 'WEPP_FILES/2016-Q2-JUL', 'WEPP_FILES/2016-Q3-SEP', 'WEPP_FILES/2016-Q4-DEC', 'WEPP_FILES/2017-Q1-MAR', 'WEPP_FILES/2017-Q2-JUL', 'WEPP_FILES/2017-Q3-SEP', 'WEPP_FILES/2017-Q4-DEC']


In [10]:
""" let's get all the WEPP dbs"""

wepps = [
    'WEPP_FILES/2007-Q4-DEC',
    'WEPP_FILES/2008-Q4-DEC',
    'WEPP_FILES/2009-Q4-DEC',
    'WEPP_FILES/2010-Q4-DEC',
    'WEPP_FILES/2011-Q3-SEP',
    'WEPP_FILES/2012-Q4-DEC',
    'WEPP_FILES/2013-Q4-DEC',
    'WEPP_FILES/2014-q4-dec',
    #'WEPP_FILES/2015-Q1-MAR',
    #'WEPP_FILES/2015-Q2-JUN',
    #'WEPP_FILES/2015-q3-sep',
    'WEPP_FILES/2015-Q4-DEC',
    #'WEPP_FILES/2016-Q1-APR',
    #'WEPP_FILES/2016-Q2-JUL',
    #'WEPP_FILES/2016-Q3-SEP',
    'WEPP_FILES/2016-Q4-DEC',
    #'WEPP_FILES/2017-Q1-MAR',
    #'WEPP_FILES/2017-Q2-JUL',
    #'WEPP_FILES/2017-Q3-SEP',
    'WEPP_FILES/2017-Q4-DEC']

years = range(2007,2018)
#print years

wepps_dict = dict(zip(years,wepps))

print wepps_dict

wepp_dfs = {}

for y in years:
    wepp_dfs[y] = get_wepp(wepps_dict[y])
    

{2016: 'WEPP_FILES/2016-Q4-DEC', 2017: 'WEPP_FILES/2017-Q4-DEC', 2007: 'WEPP_FILES/2007-Q4-DEC', 2008: 'WEPP_FILES/2008-Q4-DEC', 2009: 'WEPP_FILES/2009-Q4-DEC', 2010: 'WEPP_FILES/2010-Q4-DEC', 2011: 'WEPP_FILES/2011-Q3-SEP', 2012: 'WEPP_FILES/2012-Q4-DEC', 2013: 'WEPP_FILES/2013-Q4-DEC', 2014: 'WEPP_FILES/2014-q4-dec', 2015: 'WEPP_FILES/2015-Q4-DEC'}
xlsx ALLUNITS
csv ALLUNITS
csv ALLUNITS
csv ALLUNITS
csv ALLUNITS
csv ALLUNITS
csv ALLUNITS
csv AllUnits
csv ALLUNITS
csv ALLUNITS
csv ALLUNITS


In [11]:
""" fix all the wepp stuff, fix categories, interpolate dates, add all the columns """
def prep_wepp(wepp_df):
    # merge with ISO, country budgets and load factors 
    print '~~~~~~ GENERATING DF ~~~~~~~'
    print 'loading df...'
    df_iso = pd.read_csv('country_ISO_update.csv')
    fuel_class = 'fuel_classification_database.dta'
    df_fuel_class = pd.io.stata.read_stata(fuel_class)
    heat_rates_xls = 'Heat rates_v3.xls'
    df_heatrates = pd.read_excel(heat_rates_xls, sheet_name='CSV_output')
    df_load_factor = pd.io.stata.read_stata('load_factor_database.dta')

    print 'loaded dfs: '
    print 'merging dfs and filling missing years...'
    #df_fuel_load = pd.merge(df_fuel_class, df_load_factor, on='fuel_class')
    #print df_iso
    #print df_fuel_class
    #print df_heatrates
    #print df_load_factor
    #print list(wepp_df)
    #print wepp_df['FUEL']

    df_fuel_class.rename(columns = {'fuel': 'FUEL'}, inplace = True)
    
    #fix fuel classes
    wepp_df = wepp_df.merge(df_fuel_class, on='FUEL', how='left')
    df_wepp_em_fact = pd.read_csv('wepp_em_fact.csv')

    #merge emissions factors
    wepp_df = wepp_df.merge(df_wepp_em_fact, left_on='FUEL', right_on='fuel', how='left')

    #prepare lookup indexer
    wepp_df['FORMAT_HR'] = wepp_df.apply(lambda row: format_hr(row), axis=1)

    #standardise statuses
    wepp_df.loc[wepp_df.STATUS=='DEF', 'STATUS'] = 'PLN'
    wepp_df.loc[wepp_df.STATUS=='DEL', 'STATUS'] = 'CON'
    wepp_df.loc[wepp_df.STATUS=='UNK', 'STATUS'] = 'PLN'
    wepp_df.loc[wepp_df.STATUS=='DAC', 'STATUS'] = 'STN'

    #print  list(df_iso)

    #add ISO
    wepp_df = wepp_df.merge(df_iso[['Caps','ISO','Region']], left_on='COUNTRY', right_on='Caps', how='left')

    #fill in missing years
    all_training = wepp_df[['YEAR','fuel_class','STATUS','Region','FORMAT_HR']]
    all_training['fuel_class'] = all_training['fuel_class'].astype('category')
    all_training['STATUS'] = all_training['STATUS'].astype('category')
    all_training['Region'] = all_training['Region'].astype('category')
    all_training['FORMAT_HR'] = all_training['FORMAT_HR'].astype('category')
    all_training = pd.get_dummies(all_training[['YEAR','fuel_class','STATUS','Region','FORMAT_HR']], columns = ['fuel_class','STATUS','Region','FORMAT_HR'])

    year_train_X = all_training[all_training.YEAR.notnull()].drop('YEAR', axis=1)
    year_train_Y = all_training.loc[all_training.YEAR.notnull(),'YEAR']
    year_train_X = sm.add_constant(year_train_X)

    test_data = all_training.loc[all_training.YEAR.isnull()].drop('YEAR', axis=1)
    test_data = sm.add_constant(test_data)

    est = sm.OLS(year_train_Y, year_train_X)
    est = est.fit()

    wepp_df['YEAR_EST_FLAG'] = 0
    wepp_df.loc[wepp_df.YEAR.isnull(),'YEAR_EST_FLAG'] = 1
    wepp_df.loc[wepp_df.YEAR.isnull(),'YEAR'] = est.predict(test_data)
    


    #get heatrates
    wepp_df = wepp_df.merge(df_heatrates, left_on='FORMAT_HR', right_on='unique_id', how='left')
    wepp_df['HEATRATE'] = wepp_df.apply(lambda row: get_hr(row), axis=1)
    drop_cols = [col for col in list(wepp_df) if isinstance(col,int)]
    wepp_df.drop(drop_cols, axis=1, inplace=True)

    #get CO2 int, CCCE
    wepp_df = wepp_df.merge(df_load_factor, on='fuel_class', how='left')
    wepp_df['YEARS_LEFT'] = np.where(wepp_df['STATUS']=='OPR', wepp_df['YEAR']+40-2017, 0)
    wepp_df.YEARS_LEFT.clip(lower=0.0, inplace=True) #set min years left to 0
    print 'dfs merged and interped: '


    print 'calculating carbon and MWs...'

    wepp_df['CO2_INT'] = wepp_df['em_fact'] /2.205 * wepp_df['HEATRATE'] / 1000
    wepp_df['CCCE'] = 8760 * wepp_df['MW'] * wepp_df['YEARS_LEFT'] * wepp_df['load_factor'] * wepp_df['CO2_INT'] /1000 #tonnes 
    #wepp_df.sort_values('CCCE', inplace=True)

    #print wepp_df
    #print list(wepp_df)
    #print wepp_df.CCCE


    #print all_countries
    #exit()

    #sort WEPP
    wepp_df.sort_values('CCCE', inplace=True, ascending=False)

    wepp_df['green']=wepp_df.fuel_class.isin(['SUN','BIOGAS','WASTE','BIOOIL','WIND','BIOMASS','GEOTHERMAL'])
    wepp_df['green_MW'] = wepp_df.MW*wepp_df.green
    wepp_df['blue']=wepp_df.fuel_class.isin(['WATER','NUCLEAR'])
    wepp_df['blue_MW'] = wepp_df.MW*wepp_df.blue
    wepp_df['solar']=wepp_df.fuel_class.isin(['SUN'])
    wepp_df['solar_MW'] = wepp_df.MW*wepp_df.solar
    wepp_df['wind']=wepp_df.fuel_class.isin(['WIND'])
    wepp_df['wind_MW'] = wepp_df.MW*wepp_df.wind
    wepp_df['ff']=~wepp_df.fuel_class.isin(['SUN','BIOGAS','WASTE','BIOOIL','WIND','BIOMASS','GEOTHERMAL','WATER','NUCLEAR'])
    wepp_df['ff_MW'] = wepp_df.MW*wepp_df.ff
    
    return wepp_df

In [12]:
for y in years:
    wepp_dfs[y] = prep_wepp(wepp_dfs[y])

~~~~~~ GENERATING DF ~~~~~~~
loading df...
loaded dfs: 
merging dfs and filling missing years...


/home/lucaskruitwagen/miniconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/lucaskruitwagen/miniconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/lucaskruitwagen/miniconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

dfs merged and interped: 
calculating carbon and MWs...
~~~~~~ GENERATING DF ~~~~~~~
loading df...
loaded dfs: 
merging dfs and filling missing years...
dfs merged and interped: 
calculating carbon and MWs...
~~~~~~ GENERATING DF ~~~~~~~
loading df...
loaded dfs: 
merging dfs and filling missing years...
dfs merged and interped: 
calculating carbon and MWs...
~~~~~~ GENERATING DF ~~~~~~~
loading df...
loaded dfs: 
merging dfs and filling missing years...
dfs merged and interped: 
calculating carbon and MWs...
~~~~~~ GENERATING DF ~~~~~~~
loading df...
loaded dfs: 
merging dfs and filling missing years...
dfs merged and interped: 
calculating carbon and MWs...
~~~~~~ GENERATING DF ~~~~~~~
loading df...
loaded dfs: 
merging dfs and filling missing years...
dfs merged and interped: 
calculating carbon and MWs...
~~~~~~ GENERATING DF ~~~~~~~
loading df...
loaded dfs: 
merging dfs and filling missing years...
dfs merged and interped: 
calculating carbon and MWs...
~~~~~~ GENERATING DF ~~~~~

In [19]:
print list(wepp_dfs[2007])
print wepp_dfs[2017].loc[wepp_dfs[2017].solar_MW>0.0,['MW','green_MW','blue_MW','solar_MW','wind_MW','ff_MW']]

[u'UNIT', u'PLANT', u'COMPANY', u'MW', u'STATUS', u'YEAR', u'UTYPE', u'FUEL', u'FUELTYPE', u'ALTFUEL', u'SSSMFR', u'BOILTYPE', u'TURBMFR', u'GENMFR', u'TURBTYPE', u'SFLOW', u'SPRESS', u'STYPE', u'STEMP', u'REHEAT1', u'REHEAT2', u'PARTCTL', u'PARTMFR', u'SO2CTL', u'FGDMFR', u'NOXCTL', u'NOXMFR', u'AE', u'CONSTRUCT', u'COOL', u'RETIRE', u'CITY', u'STATE', u'COUNTRY', u'AREA', u'SUBREGION', u'ELECTYPE', u'BUSTYPE', u'COMPID', u'LOCATIONID', u'UNITID', 'fuel_class', 'Unnamed: 0', 'fuel_x', 'em_fact', 'FORMAT_HR', 'Caps', 'ISO', 'Region', 'YEAR_EST_FLAG', u'fuel_y', u'mw_from', u'mw_to', u'fueltype', u'utype', u'stype', u'unique_id', 'HEATRATE', 'load_factor', 'YEARS_LEFT', 'CO2_INT', 'CCCE', 'green', 'green_MW', 'blue', 'blue_MW', 'solar', 'solar_MW', 'wind', 'wind_MW', 'ff', 'ff_MW']
             MW  green_MW  blue_MW  solar_MW  wind_MW  ff_MW
3         0.350     0.350      0.0     0.350      0.0    0.0
25        0.059     0.059      0.0     0.059      0.0    0.0
26        0.060     0.060

In [24]:
df_centroids = pd.read_csv('country_centroids.csv').set_index('country')
#print df_centroids.get_value('TH','latitude')
print df_centroids

all_countries = df_centroids.index.values

          latitude   longitude                              name
country                                                         
AD       42.546245    1.601554                           Andorra
AE       23.424076   53.847818              United Arab Emirates
AF       33.939110   67.709953                       Afghanistan
AG       17.060816  -61.796428               Antigua and Barbuda
AI       18.220554  -63.068615                          Anguilla
AL       41.153332   20.168331                           Albania
AM       40.069099   45.038189                           Armenia
AN       12.226079  -69.060087              Netherlands Antilles
AO      -11.202692   17.873887                            Angola
AQ      -75.250973   -0.071389                        Antarctica
AR      -38.416097  -63.616672                         Argentina
AS      -14.270972 -170.132217                    American Samoa
AT       47.516231   14.550072                           Austria
AU      -25.274398  133.7

In [25]:
def country_aggregation(wepp_df, threshold, threshold_column='mw'):

    #plot companies by CCCE
    company_df = pd.DataFrame(wepp_df.CCCE.groupby(wepp_df['COMPANY']).sum())
    company_df.sort_values('CCCE', inplace=True, ascending=False)
    #print 'company_df'
    #print company_df


    for country in all_countries:
    	#for CCCE
    	#company_df[country] = wepp_df.loc[wepp_df.ISO==country,'CCCE'].groupby(wepp_df['COMPANY']).sum()

    	#for MW
    	company_df[country] = wepp_df.loc[wepp_df.ISO==country,'MW'].groupby(wepp_df['COMPANY']).sum()
        company_df[str(country)+'_green'] = wepp_df.loc[wepp_df.ISO==country,'green_MW'].groupby(wepp_df['COMPANY']).sum()
        company_df[str(country)+'_blue'] = wepp_df.loc[wepp_df.ISO==country,'blue_MW'].groupby(wepp_df['COMPANY']).sum()
        company_df[str(country)+'_solar'] = wepp_df.loc[wepp_df.ISO==country,'solar_MW'].groupby(wepp_df['COMPANY']).sum()
        company_df[str(country)+'_wind'] = wepp_df.loc[wepp_df.ISO==country,'wind_MW'].groupby(wepp_df['COMPANY']).sum()
        company_df[str(country)+'_ff'] = wepp_df.loc[wepp_df.ISO==country,'ff_MW'].groupby(wepp_df['COMPANY']).sum()


    #checksum calculation
    #print list(company_df)
    #company_df.to_csv('test_nan.csv')
    company_df.drop(labels=[np.nan,'nan_green'], axis=1, inplace=True)

    iso_col = [h for h in list(company_df) if len(h)<3]
    company_df['checksum'] = company_df[iso_col].sum(axis=1)

    company_df.sort_values('checksum', inplace=True, ascending=False)
    #print company_df
    company_df.fillna(0.0, inplace=True)
    #see how many edges we've got
    #company_df['edges'] = company_df.count(axis=1)
    #print company_df.edges.mean()  #mean about 1.4 - nice.
    
    if threshold_column=='mw':
        company_subset_df = company_df[company_df.checksum>threshold]
    elif threshold_column=='ccce':
        company_subset_df = company_df[company_df.CCCE>=threshold]

    all_ccce = company_df.CCCE.sum()
    print 'all CCCE', all_ccce
    print 'por_CCCE',company_subset_df.CCCE.sum()/float(all_ccce)
    print 'calculated carbon and MWs: '

    return company_subset_df

In [26]:
df_companies = {}
for y in years:
    df_companies[y] = country_aggregation(wepp_dfs[y],0.0,'mw')

all CCCE 114285985641.0
por_CCCE 0.999978660996
calculated carbon and MWs: 
all CCCE 131142894405.0
por_CCCE 0.999970213628
calculated carbon and MWs: 
all CCCE 148021970030.0
por_CCCE 0.999973725065
calculated carbon and MWs: 
all CCCE 172974909011.0
por_CCCE 0.999977340135
calculated carbon and MWs: 
all CCCE 190709930523.0
por_CCCE 0.999918110708
calculated carbon and MWs: 
all CCCE 212698785517.0
por_CCCE 0.999862484174
calculated carbon and MWs: 
all CCCE 228523980434.0
por_CCCE 0.99999128439
calculated carbon and MWs: 
all CCCE 246392528640.0
por_CCCE 0.99999189299
calculated carbon and MWs: 
all CCCE 262090878620.0
por_CCCE 0.999992403921
calculated carbon and MWs: 
all CCCE 280703099600.0
por_CCCE 0.999992875864
calculated carbon and MWs: 
all CCCE 293963430936.0
por_CCCE 0.999993209395
calculated carbon and MWs: 


In [88]:
#print df_companies[2009].sort_index()#.subtract(df_companies[2008])
#print df_companies[2008].sort_index()
#print df_companies[2009].subtract(df_companies[2008]) #nan - the companies didn't exist previously

iso_cols = [h for h in list(df_companies[2007]) if len(h)<3]
green_cols = [h for h in list(df_companies[2007]) if 'green' in h]
blue_cols = [h for h in list(df_companies[2007]) if 'blue' in h]
solar_cols = [h for h in list(df_companies[2007]) if 'solar' in h]
wind_cols = [h for h in list(df_companies[2007]) if 'wind' in h]
ff_cols = [h for h in list(df_companies[2007]) if 'ff' in h]
#print green_cols

yoy_g = pd.DataFrame((df_companies[2010][green_cols].sum()- df_companies[2009][green_cols].sum())/df_companies[2009][green_cols].sum(), columns=['green'])#, index=iso_cols)
yoy_g_names = pd.DataFrame.from_dict(dict(zip(green_cols, iso_cols)), orient='index').rename(index=str, columns={0:'iso2'})

#print yoy_g.join(yoy_g_names).set_index('iso2')

def join_dfs(ldf, rdf):
    return ldf.join(rdf)

new_portions = {}
#portion of new capacity is green, YOY growth rate of all
for y in range(2008,2018):
    yoy_a =pd.DataFrame((df_companies[y][iso_cols].sum()- df_companies[y-1][iso_cols].sum())/df_companies[y-1][iso_cols].sum(), columns=['yoy_all'])#, index=iso_cols)
    
    yoy_g = pd.DataFrame((df_companies[y][green_cols].sum()- df_companies[y-1][green_cols].sum())/df_companies[y-1][green_cols].sum(), columns=['yoy_green'])#, index=iso_cols)
    yoy_g_names = pd.DataFrame.from_dict(dict(zip(green_cols, iso_cols)), orient='index').rename(index=str, columns={0:'iso2'})
    yoy_g_f = yoy_g.join(yoy_g_names).set_index('iso2')
    
    yoy_b = pd.DataFrame((df_companies[y][blue_cols].sum()- df_companies[y-1][blue_cols].sum())/df_companies[y-1][blue_cols].sum(), columns=['yoy_blue'])#, index=iso_cols)
    yoy_b_names = pd.DataFrame.from_dict(dict(zip(blue_cols, iso_cols)), orient='index').rename(index=str, columns={0:'iso2'})
    yoy_b_f = yoy_b.join(yoy_b_names).set_index('iso2')
    
    yoy_s = pd.DataFrame((df_companies[y][solar_cols].sum()- df_companies[y-1][solar_cols].sum())/df_companies[y-1][solar_cols].sum(), columns=['yoy_solar'])#, index=iso_cols)
    yoy_s_names = pd.DataFrame.from_dict(dict(zip(solar_cols, iso_cols)), orient='index').rename(index=str, columns={0:'iso2'})
    yoy_s_f = yoy_s.join(yoy_s_names).set_index('iso2')
    
    yoy_w = pd.DataFrame((df_companies[y][wind_cols].sum()- df_companies[y-1][wind_cols].sum())/df_companies[y-1][wind_cols].sum(), columns=['yoy_wind'])#, index=iso_cols)
    yoy_w_names = pd.DataFrame.from_dict(dict(zip(wind_cols, iso_cols)), orient='index').rename(index=str, columns={0:'iso2'})
    yoy_w_f = yoy_w.join(yoy_w_names).set_index('iso2')
    
    yoy_ff = pd.DataFrame((df_companies[y][ff_cols].sum()- df_companies[y-1][ff_cols].sum())/df_companies[y-1][ff_cols].sum(), columns=['yoy_ff'])#, index=iso_cols)
    yoy_ff_names = pd.DataFrame.from_dict(dict(zip(ff_cols, iso_cols)), orient='index').rename(index=str, columns={0:'iso2'})
    yoy_ff_f = yoy_ff.join(yoy_ff_names).set_index('iso2')
    
    df_out = reduce(join_dfs, [yoy_a, yoy_g_f, yoy_b_f, yoy_s_f, yoy_w_f, yoy_ff_f])

    diff_a = pd.DataFrame((df_companies[y][iso_cols].sum()- df_companies[y-1][iso_cols].sum()), columns=['diff_all'])
    diff_g = pd.DataFrame((df_companies[y][green_cols].sum()- df_companies[y-1][green_cols].sum()), columns=['diff_green']).join(yoy_g_names).set_index('iso2')
    diff_b = pd.DataFrame((df_companies[y][blue_cols].sum()- df_companies[y-1][blue_cols].sum()), columns=['diff_blue']).join(yoy_b_names).set_index('iso2')
    diff_s = pd.DataFrame((df_companies[y][solar_cols].sum()- df_companies[y-1][solar_cols].sum()), columns=['diff_solar']).join(yoy_s_names).set_index('iso2')
    diff_w = pd.DataFrame((df_companies[y][wind_cols].sum()- df_companies[y-1][wind_cols].sum()), columns=['diff_wind']).join(yoy_w_names).set_index('iso2')
    diff_ff = pd.DataFrame((df_companies[y][ff_cols].sum()- df_companies[y-1][ff_cols].sum()), columns=['diff_ff']).join(yoy_ff_names).set_index('iso2')
    

    df_out['green_ratio'] = diff_g['diff_green']/diff_a['diff_all']
    df_out['blue_ratio'] = diff_b['diff_blue']/diff_a['diff_all']
    df_out['solar_ratio'] = diff_s['diff_solar']/diff_a['diff_all']
    df_out['wind_ratio'] = diff_w['diff_wind']/diff_a['diff_all']
    df_out['ff_ratio'] = diff_ff['diff_ff']/diff_a['diff_all']
    
    new_portions[y] =df_out
    #print reduce(join_dfs, [yoy_a, yoy_g_f, yoy_b_f, yoy_s_f, yoy_w_f, yoy_ff_f])
    
    #print yoy_g_f
    #print yoy_b_f
    #print yoy_s_f
    #print yoy_w_f
    #print yoy_ff_f

In [90]:
for y in range(2008,2018):
    new_portions[y].to_csv('./matrix_csvs/'+str(y)+'_new_portions.csv', encoding='utf-8')